RAG Chatbot with KV Caching ,History awareness , Knowledge Graph

Import Librarries and Packages

Recrusive character text splitter - Chunking text

OpenAi(embeddings) , Deepseek(answer generation) models

FAISS - vector db

Prompt Template for formating prompt

Pdf reader

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from PyPDF2 import PdfReader

Extracting Document Text function

In [ ]:
def get_pdf_content(documents):
    raw_text = ""

    for document in documents:
        pdf_reader = PdfReader(document)
        for page in pdf_reader.pages:
            raw_text += page.extract_text()

    return raw_text

Chunking text using RecursiveCharacterTextSplitter

In [ ]:
def get_chunks(text):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.create_documents([text])
    return chunks


Get Embeddings using openai embedding-3 model and store in FAISS

In [ ]:
def get_embeddings_openai(chunks):
    embeddings = OpenAIEmbeddings(
        api_key = 'api_key',
        model="text-embedding-3-small")
    vector_store = FAISS.from_documents(chunks, embeddings)
    return vector_store

Retrieve Top-k(=4) queries using similarity search on vector store

In [ ]:
def top_k_queries(vector_store):
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})
    return retriever

Initialising Deepseek LLM

In [ ]:
llm = ChatDeepSeek(
    api_key='api_key',
    model="deepseek-chat",
    temperature=0.2,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

Prompt Template for the LLM with History 

In [ ]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from understandingthe provided transcript context and the conversation history.
      If the context and history are insufficient, just say you don't know.

      {context}

      history : {history}

      Question: {question}
    """,
    input_variables = ['context', 'history' ,'question']
)


History Compiling function

In [ ]:
def compile_history(history):
    return "\n".join(f"{msg['role']}: {msg['content']}" for msg in history)

Entities and Relation extraction function using Prompt 

Return Entites , relations in Cypher format for neo4j

In [ ]:
def extract_entities_relation(doc_text):

     summary_prompt = PromptTemplate(
     template = '''Summarize the following text by extracting only the key entities, their types, and relationships relevant for building a knowledge graph. Provide the summary as a list of entities and their connections in a clear, concise format without any extra explanation.




           document_text = {doc_text}
        ''',

      input_variables = ['doc_text']  )

     summary = summary_prompt.invoke({"doc_text" : doc_text})
     prompt = PromptTemplate(
     template = '''Generate Cypher queries to create a knowledge graph from the following text.
                  **Return only the Cypher code as plain text with no markdown formatting, no comments, and no extra explanation.**
                  Do not include any backticks (```) or other formatting characters.

           summary = {summary}
        ''',

      input_variables = ['summary']  )

     final_prompt = prompt.invoke({"summary":summary})
     result = llm.invoke(final_prompt)
     return result.content

import streamlit

In [ ]:
import streamlit as st


Declare session variables and import neo4j libariries 

IFrame for saving the graph.html locally

In [ ]:
if 'history' not in st.session_state:
    st.session_state.history = []

if 'kv_cache' not in st.session_state:
    st.session_state.kv_cache = {}

from neo4j import GraphDatabase
from pyvis.network import Network
from IPython.display import IFrame


Setting Up Neo4j and graph function

This function saves the graph locally in a html file.

In [ ]:
driver = GraphDatabase.driver("neo4j+s://a80e37c4.databases.neo4j.io", auth=("neo4j", "epgoSPMud2qlmFJEzS9Q5amNMGpz1EI4F1YCftHVawM"))

def display_graph(cypher,html_path="graph.html"):
    with driver.session() as session:
         session.run(cypher)

    # Build and show graph
    net = Network(notebook=True)
    with driver.session() as session:
        result = session.run("MATCH (a)-[r]->(b) RETURN a, r, b")
        for record in result:
            a = record["a"]
            b = record["b"]
            r = record["r"]
            net.add_node(a.id, label=a.get("name", str(a.id)))
            net.add_node(b.id, label=b.get("name", str(b.id)))
            net.add_edge(a.id, b.id, label=r.type)

    net.show("graph.html")
    return IFrame(src=html_path, width="100%", height="650px")

Streamlit workflow

In [ ]:
st.title("PDF Chatbot")

uploaded_file = st.file_uploader("Choose a PDF file", type="pdf")
if uploaded_file is not None:
    text = get_pdf_content([uploaded_file]) #get text
    cypher = str(extract_entities_relation(text)) #create cyphers for the summary

    print(cypher)

    display_graph(cypher) #saved the graph.html locally for viewing

    split_text = get_chunks(text) #splits chunks
    st.write("PDF text split in to chunks.")
    embedding_index = get_embeddings_openai(split_text,text)
    similar_chunks = top_k_queries(embedding_index)

    st.write("embedding peformed on chunks.")
    st.write("PDF text extracted. You can now ask questions.")

    user_question = st.text_input("Ask a question about the PDF:")
    if user_question:
        key_kv_cache = user_question.strip().lower()  #strips and lowercases the key

        if key_kv_cache in st.session_state.kv_cache:  #checking if key alreadu exists
            answer = st.session_state.kv_cache[key_kv_cache] #if yes return without invovling LLMS
            # print("in cache in cache in cache")


        else: #else create new key
            retrieved_docs = similar_chunks.invoke(user_question)
            context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)

            st.session_state.history.append({"role": "user", "content": user_question}) #append the history dictionary after evey conversation

            history_string = compile_history(st.session_state.history)

            final_prompt = prompt.invoke({
                 "context": context_text,
                 "history": history_string,
                 "question": user_question
             })

            answer = llm.invoke(final_prompt) #store answer
            st.session_state.kv_cache[key_kv_cache] = answer
            st.session_state.history.append({"role": "Chatbot", "content": str(answer.content)})
            print("yeah")

             # Display the answer
        st.write(f"Answer: {answer.content}")
       
